In [1]:
# importing libs
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, Dense, GaussianNoise,Lambda,Dropout
from keras.models import Model
from keras.models import Sequential
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam,SGD
from keras import backend as K

Using TensorFlow backend.


In [2]:
# two-stage training autoencoder for the relay network
# first, S-R link traing, to determine the source encoding and relay decoding. This is done using another notebook file in the folder. The file is named "autoencoder_SRlink_ROB.ipynb"
# second, fix the source encoding and relay decoding and train the whole system, to determine the relay encoding and destination decoding
# the relay does not make hard decision, instead, it directly encodes the soft info (the output of the softmax layer)
# this is for n = 8

In [3]:
# for reproducing reslut
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(3)

In [4]:
# defining parameters
# n = n_channel 
# k = log2(M)  
M = 2**8
k = np.log2(M)
k = int(k)
n_channel = 8
R = k/n_channel
print ('M:',M,'k:',k,'n:',n_channel,'R:',R)

M: 256 k: 8 n: 8 R: 1.0


In [5]:
#generating data of size N
N = 8000
label = np.random.randint(M,size=N)

In [6]:
# creating one hot encoded vectors
data = []
for i in label:
    temp = np.zeros(M)
    temp[i] = 1
    data.append(temp)

In [7]:
# checking data shape
data = np.array(data)
print (data.shape)

(8000, 256)


In [8]:
# checking generated data with it's label
temp_check = [17,23,45,67,89,96,72,250,350]
for i in temp_check:
    print(label[i],data[i])

25 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
212 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [9]:
# obtained models from the first stage training
# load previously trained models for point to point channels
from keras.models import load_model
encoder_SR = load_model('encoder_SR.h5', custom_objects={'n_channel':n_channel})
decoder_SR = load_model('decoder_SR.h5', custom_objects={'n_channel':n_channel})

/opt/anaconda3/envs/tf_env/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [12]:
# model for encoding at the relay (soft deicison relay), and detection at the destination
input_signal_2 = Input(shape=(M,))
encoded_S = encoder_SR(input_signal_2)
# mixed SNR training from -2 to 6.5 dB
receive_SR = GaussianNoise(np.sqrt(1/(2*R*10.0**(np.random.randint(-2, 6.5) /10.0))))(encoded_S)
out_SR = decoder_SR(receive_SR) # output is soft information
# ----------------------------------------------------------------------------
input_signal_RD = Input(shape=(M,)) # input layer for R-D link 
encoded_RD = Dense(M, activation='tanh')(input_signal_RD)
encoded1_RD = Dense(n_channel, activation='linear')(encoded_RD)
encoded2_RD = Lambda(lambda x: np.sqrt(n_channel)*K.l2_normalize(x,axis=1))(encoded1_RD)
encode_RD = Model(input_signal_RD, encoded2_RD)
signal_RD = encode_RD(out_SR)
#-------------------------------------------------------------
# R-D link noise
receive_RD = GaussianNoise(np.sqrt(1/(2*R*10.0**(np.random.randint(-2, 6.5) /10.0))))(signal_RD)
# S-D link, if use the same encoder obtained from the S-R link training
receive_SD = GaussianNoise(np.sqrt(1/(2*R*10.0**(np.random.randint(-2, 6.5) /10.0))))(encoded_S)
# decoder at the destination
input_SD = Input(shape=(n_channel,))
input_RD = Input(shape=(n_channel,))
preprocess_SD = Dense(n_channel, activation='linear')(input_SD)
preprocess_RD = Dense(n_channel, activation='linear')(input_RD)
decoded_des = keras.layers.concatenate([preprocess_RD, preprocess_SD])
decoded1_des = Dense(M, activation='softmax')(decoded_des) 
decoder_D = Model(inputs = [input_SD, input_RD], outputs = decoded1_des)
decoded_D = decoder_D(inputs = [receive_SD, receive_RD])
# model at the destination
autoencoder_des = Model(input_signal_2, decoded_D)

In [13]:
adam = Adam(lr=0.001)
autoencoder_des.compile(optimizer=adam, loss='categorical_crossentropy')

In [14]:
# printing summary of layers and it's trainable parameters 
print (autoencoder_des.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 8)            2056        input_9[0][0]                    
__________________________________________________________________________________________________
gaussian_noise_7 (GaussianNoise (None, 8)            0           model_2[3][0]                    
__________________________________________________________________________________________________
model_3 (Model)                 (None, 256)          2304        gaussian_noise_7[0][0]           
__________________________________________________________________________________________________
model_7 (M

In [15]:
# traning auto encoder
hist=autoencoder_des.fit(data, data,
                epochs=80,
                batch_size=50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/80
8000/8000 [==============================] - 8s 1ms/step - loss: 4.6717
Epoch 2/80
8000/8000 [==============================] - 3s 342us/step - loss: 3.4541
Epoch 3/80
8000/8000 [==============================] - 3s 392us/step - loss: 2.3597
Epoch 4/80
8000/8000 [==============================] - 3s 369us/step - loss: 1.5474
Epoch 5/80
8000/8000 [==============================] - 3s 346us/step - loss: 1.0333
Epoch 6/80
8000/8000 [==============================] - 3s 347us/step - loss: 0.7127
Epoch 7/80
8000/8000 [==============================] - 3s 387us/step - loss: 0.5256
Epoch 8/80
8000/8000 [==============================] - 3s 343us/step - loss: 0.4284
Epoch 9/80
8000/8000 [==============================] - 3s 332us/step - loss: 0.3458
Epoch 10/80
8000/8000 [==============================] - 3s 376us/step - loss: 0.3065
Epoch 11/80
8000/8000 [==============================] 

In [16]:
# saving autoencoder phase two
# HDF5 file, you have to pip3 install h5py if don't have it
import h5py
encode_RD.save('encode_RD.h5')
decoder_D.save('decoder_D.h5')

In [17]:
# generating data for checking BLER
N = 1*10**5
test_label = np.random.randint(M,size=N)
test_data = []

for i in test_label:
    temp = np.zeros(M)
    temp[i] = 1
    test_data.append(temp)
    
test_data = np.array(test_data)

In [18]:
# checking generated data
temp_test = 6
print (test_data[temp_test][test_label[temp_test]],test_label[temp_test])

1.0 162


In [19]:
def frange(x, y, jump):
  while x < y:
    yield x
    x += jump

In [20]:
# calculating BLER
EbNodB_range = list(frange(-2,8.5,2))
bler = [None]*len(EbNodB_range)
for n in range(0,len(EbNodB_range)):
    EbNo=10.0**(EbNodB_range[n]/10.0)
    noise_std = np.sqrt(1/(2*R*EbNo))
    noise_mean = 0
    no_errors = 0
    nn = N
    noise_SR = noise_std * np.random.randn(nn,n_channel)
    encoded_signal = encoder_SR.predict(test_data) 
    # S-R link detection
    SR_signal = encoded_signal + noise_SR
    decode_output_R = decoder_SR.predict(SR_signal)
    encoded_signal_RD = encode_RD.predict(decode_output_R)
#     pred_relay_signal =  decoder_SR.predict(SR_signal)
#     pred_index_relay = np.argmax(pred_relay_signal,axis=1)
#     onehot_relay = np.zeros(M)
#     onehot_relay[pred_index_relay] = 1 # onehot transmitted by the relay
    
    noise_SD = noise_std * np.random.randn(nn,n_channel)
    noise_RD = noise_std * np.random.randn(nn,n_channel)
    SD_signal = encoded_signal + noise_SD
#     encoded_signal_RD = encoder_RD.predict(onehot_relay)
    RD_signal = encoded_signal_RD + noise_RD
    
    pred_final_signal =  decoder_D.predict([SD_signal, RD_signal])
    pred_output = np.argmax(pred_final_signal,axis=1)
    
    no_errors = (pred_output != test_label) # !! calculate the error, for 2by1 MIMO, a pair of (x1,x2)
    no_errors =  no_errors.astype(int).sum()
    bler[n] = no_errors / nn 
    print ('SNR:',EbNodB_range[n],'BLER:',bler[n])
    # use below line for generating matlab like matrix which can be copy and paste for plotting ber graph in matlab
    #print(bler[n], " ",end='')

SNR: -2 BLER: 0.58544
SNR: 0 BLER: 0.30816
SNR: 2 BLER: 0.09177
SNR: 4 BLER: 0.01198
SNR: 6 BLER: 0.00053
SNR: 8 BLER: 0.0


In [ ]:
# ploting bler curve
# save(ber, 'BLER_aotuencoder')
np.savetxt('BLER-AE-ROB.txt', bler)
import matplotlib.pyplot as plt
from scipy import interpolate
plt.plot(EbNodB_range, bler, 'bo',label='Autoencoder-relay')
plt.yscale('log')
plt.xlabel('SNR (dB)')
plt.ylabel('BLER')
plt.grid()
plt.legend(loc='upper right',ncol = 1)